In [3]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import urllib#pour récupérer les données
import bs4#pour rendre lisibles les données
import re
import time

Récupération des députés sur le site de l'Assemblée Nationale

In [2]:
def Scrap (url):
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    page = bs4.BeautifulSoup(html, "lxml")
    return page

In [4]:
def ToDf (table) :
    Listgrossiere=pd.read_html(str(table))#on transforme le tableau en liste de dataframes
    listeDf=Listgrossiere[0]#on récupère le dataframe
    return listeDf

In [38]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str(0)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
table = page.find('table')
liste=ToDf(table)
for i in range(1):
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str((i+1)*500)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
    table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
    listeDf=ToDf(table)
    liste=pd.concat([liste,listeDf])#on récupère le dataframe et on l'ajoute à la liste
    
liste=liste.reset_index()#on recrée un index afin de ne pas s'arrêter à 500 puis recommencer comme dans les pages web
listeDepute=liste[['Nom','Né le']]#on récupère les variables qui nous intéressent  

In [26]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/0000/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
listeDf=ToDf(table)

#Ce code permet de récupérer les 'href'
listelien=[]
for ele in table.findAll('a'):
  listelien.append(ele.get('href'))
listelien[1]


'/sycomore/fiche/(num_dept)/19112'

In [15]:
listeDepute.to_csv(r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\liste_depute.csv')#on met les données sur csv pour ne pas avoir à rescrapper à chaque fois

On s'intéresse ensuite à la récupération des mandats de chaque député (partie en cours).

In [150]:
def scrap_un_mandat(mandat):
    #On récupère et lave les titres des colonnes
    columns=mandat.findAll('dt')
    columns=re.sub('<[^<]+?>', '', str(columns))  
    
    #On récupère et lave les titres des lignes
    lines=mandat.findAll('dd')
    lines=re.sub('<[^<]+?>', '', str(lines))
    lines=re.sub('\t', '', str(lines))
    lines=re.sub("\n", '', str(lines))
    line=''
    
    #on s'occupe à un autre endroit du département de mort et de naissance
    depute_naiss=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore-infos-generales"})
    dep_naiss=re.findall('\((\w*-?\w*)\s', str(depute_naiss))#on récupère les départements
    dep_naiss=dep_naiss.reverse()#on connait plus les départements de mort que de naissance, s'il n'y a qu'une donnée c'est donc un département de mort a priori (sinon, l'attachement au territoire est quand même acceptable)
    
    #On met tout dans un dataframe
    columns=columns[1:-1].split(',')
    #columns.append('département de naissance')
    #columns.append('département de mort')
    """Lève trop souvent les assertionError ! Il manque souvent au moins l'un des deux, donc il ne prend pas en compte les données"""
    lines=lines[1:-1].split(',')
    #lines.append(dep_naiss)#contient un, deux ou aucun élément selon les informations
    df=pd.DataFrame([lines],columns=columns)
    return df

In [151]:
for i in range(10) :
    if i%500==1 :#pour mesurer l'avancement du scrapping
        print(i)
    page=Scrap('http://www2.assemblee-nationale.fr/'+listelien[i])
    pouf=page.find('div', {'id':"mandats_an"})
    if pouf :
        depute=pouf.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    else :#éviter les cas d'assemblée nationale constituante qui ont un autre nom que mandats_an
        depute=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    for mandat in depute :
        df = scrap_un_mandat(mandat)
        #print(df)
        
 

None
1
None
None
None
None
None
None
None
None
None
None


In [53]:
def RecupNom (page):
    nom_depute=page.find('title')#on va là où il y a le nom
    nom_depute=re.sub('<[^<]+?>', '', str(nom_depute))#on enlève les balises
    nom_depute=nom_depute.split()#on récupère les mots du titre
    mot=nom_depute[0]#on initialise au premier mot
    nom_dep=''#on initialise la variable où on va récupérer le nom
    while mot!='-' :#on prend les premiers mots qui composent le nom, avant la suite du titre
        nom_dep=nom_dep+' '+mot#on concatène le mot appartenant au nom avec les précédents
        nom_depute=nom_depute[1:]#on supprime le mot étudié
        mot=nom_depute[0]#pour passer au suivant
    return nom_dep#on retourne le mot complet. Il est surement possible de le faire beaucoup plus vite en regex, mais les miennes ne marchent pas.

In [95]:
tps1=time.time()
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(5))
listeMandat=page
liste=[]
numProbleme=[]
for i in range(5,19501) :
    if i%500==1 :#pour mesurer l'avancement du scrapping
        print(i)
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(i+7))
    #Une petite blague c'est glissé dans le site : les mandats de sénateurs sont présent dans le code html mais ils sont "cachés"
    #On les enlève donc d'abord
    pouf=page.find('div', {'id':"mandats_an"})
    if pouf :
        depute=pouf.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    else :#éviter les cas d'assemblée nationale constituante qui ont un autre nom que mandats_an
        depute=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    for mandat in depute :
        try:
            df = scrap_un_mandat(mandat)
        except AssertionError :
            numProbleme.append(i)
        else :#ne marche pas en elif ! A cause du try
            df["nom du député"]=RecupNom(page)
            if len(liste)==0: #On initialise avec le premier mandat
                liste=df  
            else:
                liste=pd.concat([liste,df])#on fusionne les mandats
print(len(numProbleme))
tps2=time.time()
print("temps d'exécution",tps2-tps1)            

KeyboardInterrupt: 

Cette proposition permet de se passer de RecupNom mais il y a encore des bugs

In [94]:
liste=[]
for i in range(36,39) : #à terme len(listelien)
    if i%10==1 :#pour mesurer l'avancement du scrapping
        print(i)
    page=Scrap('http://www2.assemblee-nationale.fr/'+listelien[i])
    #Une petite blague c'est glissé dans le site : les mandats de sénateurs sont présent dans le code html mais ils sont "cachés"
    #On les enlève donc d'abord
    pouf=page.find('div', {'id':"mandats_an"})
    if pouf :
        depute=pouf.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    else :#éviter les cas d'assemblée nationale constituante qui ont un autre nom que mandats_an
        depute=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    for mandat in depute :   
        try:
            df = scrap_un_mandat(mandat)
            print(listelien[i])

        except AssertionError :
            numProbleme.append(i)
        else :#ne marche pas en elif ! A cause du try
            df["nom du député"]=listeDepute['Nom'][i]
            if len(liste)==0: #On initialise avec le premier mandat
                liste=df  
            else:
                liste=pd.concat([liste,df])#on fusionne les mandats
        
liste.head(5)

/sycomore/fiche/(num_dept)/19121
/sycomore/fiche/(num_dept)/19058


ValueError: 5 columns passed, passed data had 7 columns

Il y des pages de députés jusqu'à 19500 sur le site.

In [91]:
liste

[]

In [99]:
liste=liste.rename(columns={' Département': 'Département', ' Groupe': 'Groupe', ' Législature': 'Législature', ' Mandat' :'Mandat'})

In [100]:
liste_dep=liste[['Département', 'Groupe', 'Législature', 'Mandat', 'Régime politique', 'nom du député']]

In [82]:
liste

[]

In [37]:
liste_dep.to_csv(r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\liste_mandat.csv')
#on met les données sur csv pour ne pas avoir à rescrapper à chaque fois

In [ ]:
len(liste)/len(numProbleme)

*Formatage des dates

In [97]:
df=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandat.csv") 
df.head()

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,nom du député
0,0,Anciens départements d'Algérie,Socialiste,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 19 février 1946,Gouvernement provisoire de la République franç...,Bachir Ben Mohamed Abddelouahab
1,0,Anciens départements d'Algérie,Unité de la République,Ire législature,Du 30 novembre 1958 au 3 juillet 1962,Cinquième République - Assemblée nationale,Robert Abdesselam
2,0,Territoire français des Affars et des Issas,Union démocratique pour la V° République,IIIe législature,Du 23 avril 1967 au 30 mai 1968,Cinquième République - Assemblée nationale,Moussa Ali Abdoulkader
3,0,Territoire français des Affars et des Issas,Union des démocrates pour la République,IVe législature,Du 23 juin 1968 au 1er avril 1973,Cinquième République - Assemblée nationale,Moussa Ali Abdoulkader
4,0,Haute-Garonne,Républicain,IVe législature,Du 18 octobre 1885 au 11 novembre 1889,Troisième République - Chambre des députés,Valentin Abeille


In [98]:
debut_mandat=[]
for dates in df["Mandat"]:
    dates=str(dates).split()
    debut_mandat.append(' '.join(dates[1:4]))
    
df["Debut Mandat"]=np.array(debut_mandat)
df.head()

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,nom du député,Debut Mandat
0,0,Anciens départements d'Algérie,Socialiste,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 19 février 1946,Gouvernement provisoire de la République franç...,Bachir Ben Mohamed Abddelouahab,21 octobre 1945
1,0,Anciens départements d'Algérie,Unité de la République,Ire législature,Du 30 novembre 1958 au 3 juillet 1962,Cinquième République - Assemblée nationale,Robert Abdesselam,30 novembre 1958
2,0,Territoire français des Affars et des Issas,Union démocratique pour la V° République,IIIe législature,Du 23 avril 1967 au 30 mai 1968,Cinquième République - Assemblée nationale,Moussa Ali Abdoulkader,23 avril 1967
3,0,Territoire français des Affars et des Issas,Union des démocrates pour la République,IVe législature,Du 23 juin 1968 au 1er avril 1973,Cinquième République - Assemblée nationale,Moussa Ali Abdoulkader,23 juin 1968
4,0,Haute-Garonne,Républicain,IVe législature,Du 18 octobre 1885 au 11 novembre 1889,Troisième République - Chambre des députés,Valentin Abeille,18 octobre 1885
